In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
%reload_ext autoreload

In [88]:
import json

with open('topic_testing/validation_set.txt', 'rb') as f:
    request = json.load(f)

In [89]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

printing ele
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-15T07:40:19.833546Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-10-15T07:40:19.834356Z", "msg": "getting feature vector from mind service"}


KeyboardInterrupt: 

In [ ]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")


In [ ]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [ ]:
pim_result = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c"}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})

In [ ]:
group

In [ ]:
topic_pim = {
}
group_result = {}
for keys in group['group'].keys():
    for seg in group['group'][keys]:
        group_result[seg['recordingId']] = keys

In [ ]:
ranked_pims = sorted([(k,v) for (k,v) in pim_result.items()], key= lambda kv: kv[1])

In [ ]:
used_topics = []
group_no = None
index = 0
for (rec_id, distance) in ranked_pims:
    if rec_id in group_result.keys():
        group_no = group_result[rec_id]
        
        if group_no not in used_topics:
            topic_pim[index] = group_no
            used_topics.append(group_no)
            index += 1

In [120]:
topic_pim

{0: '24', 1: '40', 2: '32', 3: '15'}

In [125]:
final_output = []
final_output = list(map(lambda x: group['group'][x] , topic_pim.values()))

In [126]:
final_output[0][0]

{'id': '8ad2cdf53c0c47ee97ecc9eb649af405',
 'originalText': "Ux Designers have a a lot on their plate. They research that wire user tests the list goes on and off if you're new to the game this can all feel very overwhelming well throw that here out the window because we're about to embark on a journey through the sixteenth end of the UX design process that before starting any successful uX project you'll need to be in tune with two things your user in your brain. I start with the user big question around this is what problems are they making once you can identify the problem, then you can begin to start working to towards the solution second is understanding the brain and how does the project you're are working on aligned with the branch mission and goal to summarize in this stage you wanna identify the problems facing the users as well as the core goals of the brand and see how they along as a designer you probably have a gun of a*sumptions about what the user wants or doesn't want b

In [129]:
users = []
for result in final_output:
    temp_users = []
    for seg in result:
        if seg['spokenBy'] not in temp_users:
            temp_users.append(seg['spokenBy'])
    users.append(temp_users)

In [130]:
users

[['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc'],
 ['31a3ba4761854ad9a041ddf1c4c6a1dc']]

# Testing topic level pims

## Read json Request

In [405]:
import json

with open('topic_testing/sync_eng_16_10.txt', 'rb') as f:
    request = json.load(f)

## Get Groups for the request

In [406]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

printing ele
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 1, "number of sentences": 204, "ts": "2019-10-16T12:08:19.714654Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 1, "number of sentences": 204, "ts": "2019-10-16T12:08:19.715540Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 1, "number of sentences": 204, "ts": "2019-10-16T12:08:19.716116Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 1, "number of sentences": 204, "ts": "2019-10-16T12:08:19.716672Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 1, "number of sentences": 204, "ts": "2019-10-16T12:08:19.717216Z", "msg": "computing in batches"}
{"level": "info", "fil

{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.8454449725612356, "ts": "2019-10-16T12:09:55.289905Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.8454449725612356, "ts": "2019-10-16T12:09:55.290431Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.8454449725612356, "ts": "2019-10-16T12:09:55.290961Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.8237799407628565, "ts": "2019-10-16T12:09:55.660866Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.8237799407628565, "ts": "2019-10-16T12:09:55.6615

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not."], "2019-10-16T06:51:04Z", "7e7ccbba232d411aa95ad3f244a35f40", "414facc8cb05423695948945e6876c70"], "segment1": [["Say you're in production and have a big graph.If you have to do an alter 

In [407]:
user_id_map = {}
user_id_map = {"8d6db5f7d9b74c54ba38fe710ffcaf3f":"Krishna Sai", "c66797a92e6d46ad9573926e57f7dac3":"Nisha Yadav","31a3ba4761854ad9a041ddf1c4c6a1dc":"Reagan Rewop","84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}

for groupid in group['group'].keys():
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in group['group'][groupid]:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    #print ("User", end=" ")
    print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n  ")
    print ("Keyphrases: ", end="")
    print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n\n\n\n")

Shashank Discussed 

 Text:  So the CIA, you know we shall again just to reiterate the goal is that if you want to show some output from the graph.We should kind of determine one inside and just walk backwards as a goal either so that we have something to show that this is, you know coming from the graphs and so on and so forth, you know.Yeah, so for that like so like other databases like neo4j or Neptune and all this thing up up. They don't need us to set a defined schema beforehand. So you can just directly take whatever graph we have and we can populate it but indeed in the in case of D graph we have to set initial schema. So what are the ones that have?. Yeah, we can change the schemas. That is not. Say you're in production and have a big graph.If you have to do an alter the schemer, what is the impact of that? It would be the same as if you were to alter the scheme over here for the database. You have to break our primary database and make a data table schema cell phones represent

Keyphrases: databases like neoXnumberXj or Neptune, set initial schema, set a defined schema, case of D graph, change the schemas





Krishna Sai Discussed 

 Text:  You will want to discuss the aggressiveness thing came of specific. So when you invited the clothes and we invite either to a new workspace and we do demo calls, right? We want to filter out be more aggressive and filter things and don't fall within the Mind specification within the models knowledge things like.  Select software engineering mind for new call and it's for some reason this bad transcript key phrase. All one word is always coming wrong EXO feel sort of SFU. You want to remove Sufi from the key phrase list, at least when you show on the pins that is that something that work isn't something you guys are working on is looking into yeah as that means you're part of the cases service actually rejection, but who's working was looking into that right now? No we. It became need to see if we need to prioritize this o

Keyphrases: time, Bonnie is handling the transcript watch link, stock box guys, Itll be good, partner Summit





Krishna Sai, Venkata Dikshit Discussed 

 Text:  So the results are coming much better than Norm GT.Yeah, yeah. What was making the document comparing on on on what points the car the birth? It is better than words. So he'll be sharing in next few minutes. Yeah, they are. Yeah, and is that works? We're going to push that to production. Yes. Yeah, we can we can move it. Everything is in place. It's just we need to remap in the production. 

  
Keyphrases: works, push that to production, in production, better than words, better than Norm GT





Krishna Sai Discussed 

 Text:  So I just took the I just want to be really yeah, it's offline. Did you run any thresholds on this either? I will just check. Yeah, I use the default one the least aggressive one just to start with.He's comparing the body segmentation process with the one that we have in ceiling in production as opposed

## Get PIMs for the request

In [348]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
import json
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_pims_score(req):

    #if req_data is None:
    #    lambda_payload = {"body": input_list}
    #    print (json.dumps(lambda_payload))
    #else:
    #    lambda_payload = {"body": {"request": req_data, "text_input": input_list}}
        
    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="pim",
            InvocationType="RequestResponse",
            Payload=json.dumps(req),
        )
        lambda_output = (
            invoke_response["Payload"].read().decode("utf8")
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            result = json.loads(response_body)['d2vResult'][0]['distance']
        return result
    except Exception as e:
        print (e)
    return False

In [349]:
pim_result = {}
pim_response = {}
pim_request = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV"}
pim_response = {"contextId": request["body"]["contextId"], "mindId": "01DAAYHEKY5F4E02QVRJPTFTXV", "segments": []}
temp = {}
for seg in request['body']['segments']:
    pim_request["segments"] = [seg]
    # get_pims_score({"body":pim_request})
    pim_result[seg["recordingId"]] =  get_pims_score({"body":pim_request})
    temp = seg
    temp["distance"] = pim_result[seg["recordingId"]]
    pim_response["segments"].append(temp)

'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'
'statusCode'

In [351]:
result = []
for seg in pim_response["segments"]:
    result.append( (seg["originalText"], seg["distance"], seg["recordingId"]))
result = sorted(result, key=lambda kv:kv[1])
for (text, score, segid) in result:
    print (text , " =====> ", score, segid)

There's no restrictions because no it's not necessary really hard even sequence for generally rdbms schema means different because we try to say this is of this type and we are here in force types and trying to Signal this carbonyl Etc. This is much more looser the saying this is how creation should be defined. If you want to change it you should be able to change the schema and apply somewhere.Yeah, okay. So let's dig into that a little bit. I mean some of these things can come back and bite us once you have you know, like a big graph is head is the glass used in production mirror. Yeah. It seems like we have an example they mentioned companies so they say that companies like into it and Samsung and one bot other company. They're using it for production.but.   =====>  False 7645b39903c64a7ea445d3b391882cf0
Thus the top few ones and if next whenever we add new relations we can we can just modify this Kamala change it. Okay, so you just send for this something wrong or something extra c

In [291]:
# print (str(json.dumps({"groups": group["group"], "pims": pim_response})))

## Extract topic level pims

In [352]:
import sys
sys.path.append("../")

import extract_topic_pims

In [353]:
from extract_topic_pims.main import handler

res = handler({"body":{"groups": group["group"], "pims": pim_response}}, None)
final_pims = json.loads(res)["body"]

gs_mapping:  {'a5978cb38e8444e89f00ecf9690ac92a': '0', 'afeba6c1395f479193c47c7ef45c4ade': '0', 'e897c4898785472b95063a4d06edf2e6': '0', 'ae587cb87ac84d7c8f1392d2ba2756af': '0', '956c57fe3c8c4f01a676d2065f9a9138': '0', 'ed1e9c79422642f89f6a53abadb0bc36': '0', '6289663e3b9a4f98aa301fe48ea09eaf': '0', 'fca42b69de11423c900354a4f4ccb2c9': '0', 'ab5b084ac1c0453bb84448b38f3c8f61': '0', '417a290f55c64472b9071e74a160357e': '0', '834edf6676d44f07a5d4174676110fd8': '0', '5df91e8cd5d44630b6b0271870a2ec04': '0', 'b62835485a204faa925b3d51e3d92451': '0', '9eec7bddc6ec49cba2992b879bf7a057': '24', 'b3a1cce894b74599ac2179fac21dfbed': '0', '34bd8ae4ec2b4ce29c483763167d52d9': '0', '7588bdef1d99416da300f12f2563e7d2': '0', 'e0b435804d80466aae6b96193704815a': '0', 'cb74e77622d94deca71e15c0741d9e4a': '3', '75610ce9c556403d9e44e66bf4dd109f': '3', '8296a9ac711249debd6a40b6eb7c43f6': '19', '121d64fe09e0496482842b5a6e1f8722': '3', '7645b39903c64a7ea445d3b391882cf0': '3', '86eb68e4cca74e80931653d87783694a': '3', 

In [354]:
user_id_map = {}
user_id_map = {"2f506a3d9e814de69d46a1fbf949fdc9":"Shubham","2cd90f0674f348cc922acd6b8782ba0f":"Shubham","8d6db5f7d9b74c54ba38fe710ffcaf3f":"Krishna Sai","31a3ba4761854ad9a041ddf1c4c6a1dc":"Reagan Rewop","b4a57b25de68446cac990f856d3fe4d5":"Deep Moradia","84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}

import sys
sys.path.append("../../../ai-engine/pkg/")

from graphrank.core import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess

gr = GraphRank()
tp = TextPreprocess()
gu = GraphUtils()

def get_desc(sentence):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sentence, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(graph_obj=None, input_pos_text=pos_tuple, window=4, preserve_common_words=False)
    normal_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, post_process=True)
    desc_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, descriptive=True, post_process_descriptive=True)
    desc_keyphrase = sorted(desc_keyphrase, key=lambda kv:kv[1], reverse=True)
    normal_kp = [phrase for phrase, score in normal_keyphrase]
    desc_kp = [phrase for phrase, score in desc_keyphrase]
    
    return normal_kp, desc_kp

In [356]:
for groupid in final_pims:
    user_list =[]
    seg_list = []
    keyphrase = []
    for segi in groupid:
        if segi['spokenBy'] not in user_list:
            user_list.append(segi['spokenBy'])
        seg_list.append(segi['originalText'])
        #keyphrase.append(get_desc(segi['originalText']))
    #print ("User", end=" ")
    print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
    print ("Discussed \n\n Text: ", *seg_list, "\n\n  ")
    print ("Keyphrases: ", end="")
    print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
    print ( "\n\n\n\n")

Shashank, Krishna Sai Discussed 

 Text:  Say you're in production and have a big graph.If you have to do an alter the schemer, what is the impact of that? It would be the same as if you were to alter the scheme over here for the database. You have to break our primary database and make a data table schema cell phones representation of data now doesn't really need to make sure variable in graph terminology. What is the schema change?.  An example of a schema change leader of like what is it like it just in general for girls like what what?.  For example today we have something like to today's structure of graph will have context instance ID. Then it has a relation has meeting connects to instance ID then has segment which has like list of all the segments ideas, right? And so so our current schema will be like indexed on the segment IDs and then we will put so we will show inside segment ID. They have like all these. Spoken text and segment keywords that so I'll put an index on the key

Keyphrases: small change article, reason that timestamp, shot one example is today, reason in your calculation, incorrect in the full transcript





Krishna Sai, Nisha Yadav Discussed 

 Text:  Select software engineering mind for new call and it's for some reason this bad transcript key phrase. All one word is always coming wrong EXO feel sort of SFU. You want to remove Sufi from the key phrase list, at least when you show on the pins that is that something that work isn't something you guys are working on is looking into yeah as that means you're part of the cases service actually rejection, but who's working was looking into that right now? No we. It became need to see if we need to prioritize this over that let's prioritize what we discussed which is you know, like you it is all of these things right? Let's get the chapter key phases. Why did we deploy the champ tank efficient production in chapter using key free service.Is there a reason we didn't apply or there was some wronged 

In [355]:
user_id_map = {}
user_id_map = {"8d6db5f7d9b74c54ba38fe710ffcaf3f":"Krishna Sai", "c66797a92e6d46ad9573926e57f7dac3":"Nisha Yadav","31a3ba4761854ad9a041ddf1c4c6a1dc":"Reagan Rewop","84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}


# testing hierarchy community

In [101]:
import json

with open('topic_testing/podcast_28.txt', 'rb') as f:
    request = json.load(f)

In [102]:
from main import handler

res = handler(request, None)
group = json.loads(res['body'])

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.314253Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315016Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:02:28.315649Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.316552Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.317144Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.323420Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:02:28.325038Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people understand that subject or not.\\\\\", \\\\\"We just want it to be technically correct.\\\\\", \\\\\"So if you look up for the API definition o

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.326537Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327136Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.327728Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:02:28.328450Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.271674Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.272984Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:04:06.273614Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 4595271"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:04:06.274688Z", "msg": "Response headers: {'Date': 'Mon

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [105]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
#m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [106]:
groups = []
for i in group['group'].keys():
    if len(group['group'][i])==1:
        continue
    else:
        temp = []
        for seg in group['group'][i]:
            temp.append(seg['originalText'])
        groups.append(' '.join(temp))

In [107]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01daaqyn9gbebc92aywnxedp0c", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr = None
for segments_id in group['group'].keys():
    if len(group['group'][segments_id]) > 2:
        group_request['segments'] = group['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833136Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.833949Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T15:16:21.834604Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.835641Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.836270Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.841634Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T15:16:21.842960Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": [\\\\\"Everyone they share back again with another video.\\\\\", \\\\\"We will talk about what is a pi or also known as Catholic.\\\\\", \\\\\"Indeed everyone they share back again with another video.\\\\\", \\\\\"Now in this video we will talk about what is a pi or also known as application programming interface.\\\\\", \\\\\"Now, this is a very common term very useful term.\\\\\", \\\\\"But there is a problem with our society or our computer Society be give XnumberXfXnumberX, whether people underst

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.844338Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845103Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.845809Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T15:16:21.846798Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T15:17:31.608069Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01daaqyn9gbebc92aywnxedp0c/invocations HTTP/1.1\" 200 2153774"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.609441Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '2153774', 'Connection': 'keep-alive', 'x-amzn-RequestId': '326d4a56-be3d-4ab5-8724-72d3bb2f89af', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d921c6f-7cdeec60f9fc47486cd2f448;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T15:17:31.610493Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 15:18:13 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic."], "2019-09-28T05:34:40Z", "31a3ba4761854ad9a041ddf1c4c6a1dc", "4e865bf0f59b440db31c432cf51347b7"], "segment1": [["Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and a

In [108]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
m_time = formatTime("2019-09-28T05:34:21Z", True)
for i in group_itr['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:00:19 4e865bf0f59b440db31c432cf51347b7 

Everyone they share back again with another video. Now in this video. We will talk about what is a pi or also known as Catholic. 

Minutes from the start of the meeting:  0:00:58 a0349078dbc746199b273b5a3b3e8f51 

Indeed everyone they share back again with another video. Now in this video we will talk about what is a pi or also known as application programming interface. Now, this is a very common term very useful term. But there is a problem with our society or our computer Society be give 0f2, whether people understand that subject or not. We just want it to be technically correct. So if you look up for the API definition on any platform Wikipedia blog, they are technically correct. That's one part when they are not so much understandable. So in this video we are going to make sure that you understand and API what else API and after watching this video you never ever watch a

In [89]:
group_result = {}
#group_response = {}
group_request = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "instanceId": "xyz"}
#group_response = {"contextId": request["body"]["contextId"], "mindId": "01DADP74WFV607KNPCB6VVXGTG", "segments": []}
temp = {}
group_itr_2 = None
for segments_id in group_itr['group'].keys():
    if len(group_itr['group'][segments_id]) > 2:
        group_request['segments'] = group_itr['group'][segments_id]
        res = handler({"body":group_request}, None)
        group_itr_2 = json.loads(res['body'])
        break

{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.140392Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.141156Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 21, "module": "scorer", "ts": "2019-09-30T14:29:07.142037Z", "msg": "getting feature vector from mind service"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.142868Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.143833Z", "msg": "Event before-parameter-build.lambda.Invoke: calling handler <function generate_idempotent_uuid at 0x7f97bfd1af80>"}
{"level": "debug", "filename"

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.149214Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "endpoint.py", "lineno": 101, "module": "endpoint", "ts": "2019-09-30T14:29:07.150331Z", "msg": "Making request for OperationModel(name=Invoke) with params: {'url_path': '/2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations', 'query_string': {}, 'method': 'POST', 'headers': {'X-Amz-Invocation-Type': 'RequestResponse', 'User-Agent': 'Boto3/1.9.197 Python/3.7.4 Linux/5.2.11-arch1-1-ARCH Botocore/1.12.197'}, 'body': b'{\"body\": \"{\\\\\"text\\\\\": \\\\\" Okay I think my join let us just started with that community on the language models right yep so. Dont how any with having but engine General communities. Yeah, it is was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally does not behavior.

{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.151367Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152272Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.152928Z", "msg": "Event request-created.lambda.Invoke: calling handler <bound method RequestSigner.handler of <botocore.signers.RequestSigner object at 0x7f97bfabebd0>>"}
{"level": "debug", "filename": "hooks.py", "lineno": 210, "module": "hooks", "ts": "2019-09-30T14:29:07.153688Z", "msg": "Event choose-signer.lambda.Invoke: calling handler <f

{"level": "debug", "filename": "connectionpool.py", "lineno": 396, "module": "connectionpool", "ts": "2019-09-30T14:29:35.851920Z", "msg": "https://lambda.us-east-1.amazonaws.com:443 \"POST /2015-03-31/functions/mind-01dadp74wfv607knpcb6vvxgtg/invocations HTTP/1.1\" 200 3042138"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.853292Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '3042138', 'Connection': 'keep-alive', 'x-amzn-RequestId': '4eb66edf-ff05-40af-9613-4a7d8e93c0d7', 'x-amzn-Remapped-Content-Length': '0', 'X-Amz-Executed-Version': '$LATEST', 'X-Amzn-Trace-Id': 'root=1-5d92115d-b957f7527c2b1c3c87c74ea0;sampled=0'}"}
{"level": "debug", "filename": "parsers.py", "lineno": 234, "module": "parsers", "ts": "2019-09-30T14:29:35.855192Z", "msg": "Response headers: {'Date': 'Mon, 30 Sep 2019 14:30:17 GMT', 'Content-Type': 'application/json', 'Conten

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

{"level": "info", "filename": "grouper_segments.py", "lineno": 419, "module": "grouper_segments", "PIMs": {"3": {"segment0": [["Okay I think my join let's just started with that community on the language models right yep so."], "2019-09-30T10:29:13Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "ecc525a1a4d34f53aeaf2d97feada15e"], "segment1": [["Don't how any with having but engine General communities."], "2019-09-30T10:29:30Z", "b1e8787a9a1f4859ac11cbb6a8124fd9", "069ed6f0a2434222a5fc87c90c6dbe28"], "segment2": [["Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah."], "2019-09-30T10:29:49Z", "ecfeeb757f0a4d47af1ebd513929264a", "74cf907c8c1d473abb406d002d35c3a9"], "segment3": [["Brand able to distinguis

In [90]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:28:00Z", True)
for i in group_itr_2['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group_itr_2['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  3


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98

In [75]:
## visualization
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True)
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
m_time = formatTime("2019-09-30T10:08:00Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  0:21:13 ecc525a1a4d34f53aeaf2d97feada15e 

Okay I think my join let's just started with that community on the language models right yep so. 

Minutes from the start of the meeting:  0:21:30 069ed6f0a2434222a5fc87c90c6dbe28 

Don't how any with having but engine General communities. 

Minutes from the start of the meeting:  0:21:49 74cf907c8c1d473abb406d002d35c3a9 

Yeah, it's was giving good results. I just few meetings so in that I also observe that we are able to say generated good groups are coming up and the general call the development. The segments go with instances as code low so by was paying outfit with as a sentence level and I group the scale of four changes but generally doesn't behavior. Yeah. 

Minutes from the start of the meeting:  0:22:27 2c1b3cd9bfde44af956e7c0c7b3716fd 

Brand able to distinguish domain out of the and that case. 

Minutes from the start of the meeting:  0:22:33 78492bd0a4eb4de1810c98